# Model training and Evaluation

In [241]:
import numpy as np
import pandas as pd

import librosa
import librosa.display

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

## Data loading

The preprocessed data from the previous notebook is loaded.

In [2]:
%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r label_encoder

## 1. K-Nearest Neighbors

In the first place, a K-Nearest Neighbor algorithm is tested. In order to find k, GridSearch is used with a series of parameters to evaluate. Using GridSearch we can evaluate all the possible combinations of the hyperparameters values using cross-validation.

In [176]:
#Defining grid parameters for the algorithm to test

grid_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

In [177]:
model_knn = GridSearchCV(KNeighborsClassifier(), grid_params, scoring='accuracy',cv=5)
model_knn.fit(x_train, y_train)

/home/diego/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [3, 5, 7, 9, 11, 15], 'metric': ['euclidean', 'manhattan'], 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

We can check the accuracy of our model and also the confusion matrix

In [196]:
#Model score: Accuracy
knn_score = model_knn.score(x_test, y_test)
print ("Model Score",  knn_score)

y_predict_knn = model_knn.predict(x_test)

#Confusion matrix
conf_matrix = confusion_matrix(y_predict_knn, y_test)
conf_matrix

('Model Score', 0.535)


array([[15,  0,  2,  1,  0,  1,  1,  0,  0,  0],
       [ 1, 12,  0,  0,  0,  2,  0,  0,  0,  1],
       [ 0,  0, 14,  2,  2,  1,  1,  0,  2,  3],
       [ 0,  0,  2,  9,  2,  1,  0,  2,  3,  2],
       [ 1,  0,  1,  3,  3,  0,  2,  1,  4,  1],
       [ 1,  1,  3,  0,  0, 13,  0,  1,  2,  0],
       [ 1,  0,  1,  1,  2,  1, 20,  0,  0,  1],
       [ 0,  0,  1,  1,  0,  0,  0,  6,  1,  1],
       [ 0,  0,  2,  2,  6,  1,  0,  2,  8,  5],
       [ 1,  0,  1,  2,  0,  2,  1,  1,  3,  7]])

In [197]:
#Best parameters found by GridSearch
model_knn.best_params_

{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}

It can be observed that the first model does not perform as we expected, with an accuracy of 0.53. Let's try with another models before changing some parameters in the preprocessing notebook.

## 2. Decision Tree

We will use a simple Decision Tree classifier.

In [222]:
# Train a decision tree model

model_tree = DecisionTreeClassifier(random_state=10)
model_tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best')

Now, we are going to predict the labels for our test data

In [223]:
y_predict_tree = model_tree.predict(x_test)

#class_rep_tree = classification_report(y_test, predict_labels_tree)
conf_matrix_tree = confusion_matrix(y_predict, y_test)
conf_matrix_tree


#print("Decision Tree:", class_rep_tree)

array([[11,  1,  6,  1,  0,  1,  1,  0,  0,  4],
       [ 0, 11,  3,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  8,  0,  1,  4,  0,  0,  4,  0],
       [ 1,  0,  2,  5,  2,  1,  1,  2,  2,  3],
       [ 0,  0,  1,  3,  2,  1,  2,  0,  6,  3],
       [ 3,  1,  2,  0,  0, 10,  0,  0,  2,  3],
       [ 2,  0,  1,  2,  2,  0, 18,  0,  2,  1],
       [ 0,  0,  0,  1,  2,  0,  0,  8,  1,  0],
       [ 0,  0,  0,  3,  5,  2,  1,  1,  5,  1],
       [ 3,  0,  4,  6,  1,  3,  2,  2,  1,  6]])

In [228]:
print("Accuracy:",metrics.accuracy_score(y_test, y_predict_tree))

('Accuracy:', 0.42)


We can observe that Accuracy has not improved in this case.

## 3. Random Forests

Let's see what can we do with a Random Forest model.

In [231]:
model_forest = RandomForestClassifier(n_estimators=100, max_depth= 5)
model_forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [233]:
model_forest.score (x_test, y_test)

0.43

In [239]:
y_predict_forest = model_forest.predict(x_test)
conf_matrix = confusion_matrix(y_test, y_predict_forest)
print(conf_matrix)
print(classification_report(y_test, y_predict_forest))

[[ 8  1  0  0  0  1  6  0  0  4]
 [ 0 13  0  0  0  0  0  0  0  0]
 [ 7  1  4  2  1  5  1  2  0  4]
 [ 0  0  0  9  3  0  1  5  1  2]
 [ 0  0  0  0  3  0  2  3  5  2]
 [ 5  1  3  2  0  5  1  1  4  0]
 [ 1  0  0  0  1  0 22  0  0  1]
 [ 0  0  0  2  0  0  0  8  3  0]
 [ 1  1  1  2  3  0  2  2  9  2]
 [ 4  0  1  4  0  0  1  2  4  5]]
              precision    recall  f1-score   support

           0       0.31      0.40      0.35        20
           1       0.76      1.00      0.87        13
           2       0.44      0.15      0.22        27
           3       0.43      0.43      0.43        21
           4       0.27      0.20      0.23        15
           5       0.45      0.23      0.30        22
           6       0.61      0.88      0.72        25
           7       0.35      0.62      0.44        13
           8       0.35      0.39      0.37        23
           9       0.25      0.24      0.24        21

   micro avg       0.43      0.43      0.43       200
   macro avg       

We achieved a similar accuracy to the obtained in the Decision Tree model. As a Random Forest is a more robust algorithm, we can start thinking we can improve our preprocessing stage to improve this metrics.

## 4. Logistic Regression

Finally, we will compare the performance of our models to a Logistic Regression:

In [242]:
model_logreg = LogisticRegression(random_state=10)
model_logreg.fit(x_train, y_train)

/home/diego/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/diego/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=10, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [245]:
y_predict_logreg = model_logreg.predict(x_test)
print(classification_report(y_test, y_predict_logreg))

              precision    recall  f1-score   support

           0       0.27      0.50      0.35        20
           1       0.81      1.00      0.90        13
           2       0.53      0.33      0.41        27
           3       0.42      0.38      0.40        21
           4       0.31      0.27      0.29        15
           5       0.58      0.32      0.41        22
           6       0.71      0.88      0.79        25
           7       0.47      0.69      0.56        13
           8       0.36      0.35      0.36        23
           9       0.29      0.19      0.23        21

   micro avg       0.47      0.47      0.47       200
   macro avg       0.48      0.49      0.47       200
weighted avg       0.47      0.47      0.46       200



Similar to the previous models, we have an average of precision of 0.47. We can try to make some modifications to the preprocessing (more number of mel coefficients for example, as we are using only 12 and the default is 20) to see if we can achieve better metrics.